# Procesamiento de pdfs

# columna id, partido politico,content

## 1 pag useless

In [10]:
import os
import csv
from PyPDF2 import PdfReader
import sys

# Incrementar el límite del tamaño del campo
csv.field_size_limit(10**6)

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras 3 páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=3):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")



Datos agregados al archivo CSV: candidatos.csv


## 2 pag useless

In [11]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf",
    "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf",
    "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf",
    "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf",
    "PARTIDO AVANZA _Plan de trabajo_.pdf",
    "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=2):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


## 3 pag useless

In [ ]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf",
    "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=3):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


## 4

In [14]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf",
    "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf",
    "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf",
    "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=4):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 5

In [15]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=5):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 7

In [16]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=7):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 8

In [50]:
import os
import re
import pandas as pd
import pdfplumber

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de diccionarios específicos a procesar
file_parameters = [
    {"file_name": "REVOLUCIÓN CIUDADANA - RETO _Plan de trabajo_.pdf", "exclude_pages_start": 8},
    {"file_name": "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf", "exclude_pages_start": 7},
    {"file_name": "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf","exclude_pages_start": 5},
    {"file_name": "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO AVANZA _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf", "exclude_pages_start": 1}
]

# Función para obtener el último ID del archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    df = pd.read_csv(csv_path, sep="|", encoding="utf-8")
    if df.empty:
        return 1
    return df['ID'].iloc[-1] + 1

# Función para extraer texto del PDF excluyendo las primeras y últimas páginas
def extract_text_excluding_pages(pdf_path, exclude_pages_start, exclude_pages_end=1):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(exclude_pages_start, len(pdf.pages) - exclude_pages_end):
            page_text = pdf.pages[i].extract_text()
            if page_text:
                extracted_text += page_text + "\n"
    return extracted_text.strip()

# Función para limpiar el contenido del texto
def clean_content(text):

    # Eliminar viñetas comunes
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    text = re.sub(r'\(cid:\d+\)', '', text)
    # Eliminar enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Al inicio de la línea
    text = re.sub(r'\n\d+\.', '\n', text)  # En medio del texto
    
    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Crear una lista para almacenar los datos
data = []

# Recorrer la lista de diccionarios específicos
for file_param in file_parameters:
    file_name = file_param["file_name"]
    exclude_pages_start = file_param["exclude_pages_start"]

    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)

    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")

        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path, exclude_pages_start=exclude_pages_start)

        # Limpiar el contenido extraído
        cleaned_content = clean_content(content)

        # Agregar los datos a la lista
        data.append([file_id, processed_name, cleaned_content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Crear un DataFrame a partir de los datos nuevos
df_new = pd.DataFrame(data, columns=['ID', 'Nombre', 'Contenido'])

# Verificar si el archivo CSV ya existe
if os.path.exists(output_csv):
    # Leer el archivo CSV existente
    df_existing = pd.read_csv(output_csv, sep="|", encoding="utf-8")
    # Concatenar los datos nuevos con los existentes
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    df_combined = df_new

# Guardar el DataFrame combinado en el archivo CSV con delimitador ";"
df_combined.to_csv(output_csv, sep=";", index=False, encoding="utf-8")

print(f"Datos agregados al archivo CSV: {output_csv}")

Datos agregados al archivo CSV: candidatos.csv


In [67]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('candidatos.csv', sep=';')

# Buscar la fila donde el valor de la columna 'id' sea 11
fila = df[df['ID'] == 2]

# Mostrar la fila
print(fila)

   ID                                    Nombre  \
1   2  PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA    

                                           Contenido  
1  Introducción En una economía globalizada y alt...  


no tiene content 11,13,15